## Indice
#### 1. Pulizia crawled data
#### 2.Creo una colonna con i tag puliti
#### 3.Creazione network di partenza
#### 4.Creazione dataset da 10k e 3k nodi
qui però stiamo selezionando i primi 10k e 3k quindi stiamo proprio tagliando il dataset, questo potrebbe essere un problema dato che potrebbe non essere rappresentativo di tutto il dataset
###### 4.1 Controllo preliminare dei nodi ed edges
#### 5. Risultati

In [1]:
import twint
import pandas as pd


import networkx as nx
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import powerlaw
warnings.filterwarnings('ignore')


%matplotlib inline 

### PULIZIA CRAWLED DATA

In [2]:
df = pd.read_csv("../Dataset FFF/climatechange.csv", sep='\t')

In [3]:
type(df)

pandas.core.frame.DataFrame

In [4]:
#riduco le dimensioni del dataframe per avvicinarcio ai 15k, lo faccio togliendo parte finale dei dati 
df = df.iloc[:41700]

In [5]:
df.tail(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
41698,1187693451556610049,1187693451556610049,2019-10-25 13:32:39 ora legale Europa occidentale,2019-10-25,13:32:39,200,712989553,just_aenna,Anna Hübner,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
41699,1187693166742429696,1187693166742429696,2019-10-25 13:31:31 ora legale Europa occidentale,2019-10-25,13:31:31,200,68459659,winterbe_,Benjamin,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
# Get all names 
for col_name in df.columns: 
    print(col_name)

id
conversation_id
created_at
date
time
timezone
user_id
username
name
place
tweet
language
mentions
urls
photos
replies_count
retweets_count
likes_count
hashtags
cashtags
link
retweet
quote_url
video
thumbnail
near
geo
source
user_rt_id
user_rt
retweet_id
reply_to
retweet_date
translate
trans_src
trans_dest


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41700 entries, 0 to 41699
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               41700 non-null  int64  
 1   conversation_id  41700 non-null  int64  
 2   created_at       41700 non-null  object 
 3   date             41700 non-null  object 
 4   time             41700 non-null  object 
 5   timezone         41700 non-null  int64  
 6   user_id          41700 non-null  int64  
 7   username         41700 non-null  object 
 8   name             41697 non-null  object 
 9   place            12 non-null     object 
 10  tweet            41700 non-null  object 
 11  language         41700 non-null  object 
 12  mentions         41700 non-null  object 
 13  urls             41700 non-null  object 
 14  photos           41700 non-null  object 
 15  replies_count    41700 non-null  int64  
 16  retweets_count   41700 non-null  int64  
 17  likes_count 

Toglie created_at, time, timezone, lasciando la data
Sono dati ripetuti 

In [3]:
df.drop(['trans_dest', 'trans_src','translate','retweet_date','geo','source','user_rt_id', 'user_rt','retweet_id',
        'near','place','thumbnail', 'time', 'timezone', 'video', 'photos', 'cashtags', 'created_at','quote_url','id','conversation_id'], axis=1, inplace=True)





In [9]:
#No valori nulli!
df['language'].isnull().values.any()

False

In [10]:
Lingue=list(set(df['language']))
#Lingue

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41700 entries, 0 to 41699
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            41700 non-null  object
 1   user_id         41700 non-null  int64 
 2   username        41700 non-null  object
 3   name            41697 non-null  object
 4   tweet           41700 non-null  object
 5   language        41700 non-null  object
 6   mentions        41700 non-null  object
 7   urls            41700 non-null  object
 8   replies_count   41700 non-null  int64 
 9   retweets_count  41700 non-null  int64 
 10  likes_count     41700 non-null  int64 
 11  hashtags        41700 non-null  object
 12  link            41700 non-null  object
 13  retweet         41700 non-null  bool  
 14  reply_to        41700 non-null  object
dtypes: bool(1), int64(4), object(10)
memory usage: 4.5+ MB


Restingendo il range di tweet con numero di likes, retweet e replies
Si arriva a 34848 valori totali

In [4]:
df=df[df['likes_count']>=10]
df=df[df['replies_count']>=3]
df=df[df['retweets_count']>=3]

In [5]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19579 entries, 6 to 50015
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            19579 non-null  object
 1   user_id         19579 non-null  int64 
 2   username        19579 non-null  object
 3   name            19571 non-null  object
 4   tweet           19579 non-null  object
 5   language        19579 non-null  object
 6   mentions        19579 non-null  object
 7   urls            19579 non-null  object
 8   replies_count   19579 non-null  int64 
 9   retweets_count  19579 non-null  int64 
 10  likes_count     19579 non-null  int64 
 11  hashtags        19579 non-null  object
 12  link            19579 non-null  object
 13  retweet         19579 non-null  bool  
 14  reply_to        19579 non-null  object
dtypes: bool(1), int64(4), object(10)
memory usage: 2.3+ MB


### Creo una colonna con i tag puliti

In [6]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mir-9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Faccio questa funzione per pulire i tags dalle quadre, la applico a una nuova colonna (clean_tags)

In [7]:
def remove_content(text):
    text = re.sub(r"\[", "", text) #remove ]
    text=re.sub(r'\]','',text) #remove ]
    text=re.sub(r'\'','',text) #remove apici
    text=re.sub(r'\n','',text) #remove spazio vuoto

    return text

In [8]:
len(df['hashtags'])

19579

In [9]:
df['clean_tags']=df['hashtags'].apply(lambda x: remove_content(x))
len(df['clean_tags'])

19579

In [10]:
df.insert(0, 'clean_tags', df.pop('clean_tags'))

In [11]:
df.insert(1, 'hashtags', df.pop('hashtags'))

In [12]:
df.head(1)

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,urls,replies_count,retweets_count,likes_count,link,retweet,reply_to
6,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[]


In [13]:
#Salvo le modifiche nel df nuovo
df.to_csv('../Dataset FFF/fff.csv', index=False)

###  Creazione network di partenza

In [14]:
df1=pd.read_csv("../Dataset FFF/fff.csv")
df1.head(1)

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,urls,replies_count,retweets_count,likes_count,link,retweet,reply_to
0,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[]


In [15]:
#Ti crea un dizionario in cui le chiavi sono i tag singoli e i valori sono gli id degli user
tags_dict = {}
for i in range(len(df1)):
    ll = str(df1["clean_tags"][i]).split(",")[:-1]
    for tg in ll:
        if tg not in tags_dict:
            tags_dict[tg] = [df1["user_id"][i]]
        else:
            tags_dict[tg].append(df1["user_id"][i])

In [16]:
#Crea la lista di tags dalle chiavi del dizionario e li associa alla variabile tag
tags = list(tags_dict.keys())
#crea una lista costituita da coppie di tags
couple_tags = []
for i in range(len(tags)):
    for j in tags[i+1:]:
        couple_tags.append((tags[i], j)) 

In [17]:
couple_tags

[('klimakrise', ' govegan'),
 ('klimakrise', 'fridaysforfuture'),
 ('klimakrise', 'climatecrisis'),
 ('klimakrise', ' climatestrike'),
 ('klimakrise', ' climateaction'),
 ('klimakrise', ' climate'),
 ('klimakrise', ' savesoilbengaluru'),
 ('klimakrise', ' fridaysforfuture'),
 ('klimakrise', 'peoplepower'),
 ('klimakrise', ' actnow'),
 ('klimakrise', ' climatecrisis'),
 ('klimakrise', ' climateemergency'),
 ('klimakrise', ' mapa'),
 ('klimakrise', 'bangladesh'),
 ('klimakrise', ' climatejustice'),
 ('klimakrise', ' peoplenotprofit'),
 ('klimakrise', 'climatestrike'),
 ('klimakrise', ' plastic'),
 ('klimakrise', ' humanity'),
 ('klimakrise', ' recycled'),
 ('klimakrise', ' plasticwaste'),
 ('klimakrise', ' planet'),
 ('klimakrise', 'notmytaxonomy'),
 ('klimakrise', 'laufzeitverlängerung'),
 ('klimakrise', ' akw'),
 ('klimakrise', 'climatestrikeonline'),
 ('klimakrise', 'schoolstrike4climate'),
 ('klimakrise', ' climatejusticenow'),
 ('klimakrise', ' climateactionnow'),
 ('klimakrise', ' 

In [18]:
len(couple_tags)

27302355

In [19]:
len(tags)

7390

In [28]:
len(tags)

11172

In [20]:
source = []
target = []
count = []
nodes = []
for i in couple_tags:
    ll = list(set(tags_dict[i[0]]) & set(tags_dict[i[1]]))
    if len(ll)>0:
        source.append(i[0])
        target.append(i[1])
        count.append(len(ll))
        nodes.append(i[0])
        nodes.append(i[1])

In [21]:
nodes = list(set(nodes)) #list()crea una lista; set(nodes) prende i nodi senza ripetizioni

In [22]:
len(nodes)

7241

In [32]:
nodes

[' todoslossantos',
 'ggultras',
 ' ees20',
 'queenelizabeth',
 ' massextinction',
 ' cancún',
 ' workshop',
 ' lichtaus',
 ' wetsuwetensolidarity',
 ' ilfordfp5',
 ' razetozero',
 ' oil',
 ' davos2020',
 ' kemmerich',
 ' endcoal',
 ' katebernadette',
 'kesselbambule',
 ' gruene',
 'autoprämie',
 'pollutersout',
 ' futures',
 ' immigration',
 'coronapandemie',
 ' stayathome',
 ' indymedia',
 ' hüpfdeutsche',
 'neckarwestheim',
 ' scioperoperilclima',
 ' gegendiea49',
 'lausanne',
 ' ambiente',
 'iklimiçinokulgrevi',
 'union',
 ' ge2019',
 'elmshorn',
 ' bs2911',
 ' cambiamenticlimatici',
 ' linken',
 'transpride',
 'unteilbar',
 ' europawahl',
 'mumbai',
 ' xmas21',
 ' ratifyparis',
 ' brasilien',
 'climatereporting',
 ' plasticstreaty',
 'katyayan',
 ' wastetoenergy',
 ' wo0606',
 ' cleanup',
 ' tiktokdance',
 'fossilfriday',
 'globalstrike',
 'filme',
 'polizeigewalt',
 ' riberasacra',
 'stopp',
 ' connewitz',
 'strongertogether',
 ' jusos',
 ' kxlisdead',
 ' yourself',
 ' fightforev

In [23]:
new_df = pd.DataFrame({"Id": nodes})
new_df.to_csv("../Dataset FFF/nodes.csv", index=False)

In [24]:
new_df = pd.DataFrame({"Source": source,
                       "Target": target,
                      "Count": count})
new_df.to_csv("../Dataset FFF/edges.csv", index=False)

In [25]:
others_df = pd.DataFrame({"Id": nodes})
others_df.to_csv("../Dataset FFF/nodes.csv", index=False)

In [26]:
others_df = pd.DataFrame({"Source": source,
                       "Target": target,
                      "Count": count})
others_df.to_csv("../Dataset FFF/edges.csv", index=False)

### Creazione dataset da 10k e 3k nodi

###### Controllo preliminare dei nodi ed edges

qui però stiamo selezionando i primi 10k e 3k quindi stiamo proprio tagliando il dataset, questo potrebbe essere un problema dato che potrebbe non essere rappresentativo di tutto il dataset


In [27]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import math
import numpy as np
import networkx as nx
from operator import itemgetter

In [28]:
path = "../Dataset FFF/edges.csv"
path_all = "../Dataset FFF/fff.csv"
path_nodes = "../Dataset FFF/nodes.csv"

In [29]:
fff = pd.read_csv(path)

In [30]:
fff.sort_values('Count', ascending=False)

,Source,Target,Count
1075,fridaysforfuture,fridaysforfuture,438
6057,climatestrike,fridaysforfuture,330
10804,fridaysforfuture,climatestrike,253
1072,fridaysforfuture,climatestrike,232
8285,climateaction,fridaysforfuture,174
...,...,...,...
78119,eu,cop25,1
78120,eu,klimahysterie,1
78121,eu,city,1
78122,eu,hitzesommer,1


In [31]:
G = nx.read_edgelist(path, comments = "S", delimiter = ",", data=(("Count", int),))

In [32]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')
sorted_degree = sorted(degree_dict.items(), key = itemgetter(1), reverse = True)
print(" ---- Top 20 nodes by degree ----")
for d in sorted_degree[:20]:
    print("\t" + str(d))

 ---- Top 20 nodes by degree ----
	('fridaysforfuture', 6208)
	('climatestrike', 2852)
	('climatecrisis', 1866)
	('climateaction', 1805)
	('klimaschutz', 1643)
	('climateemergency', 1594)
	('klimastreik', 1533)
	('climatejustice', 1503)
	('klimakrise', 1454)
	('climatechange', 1423)
	('fridayforfuture', 1314)
	('climatestrikeonline', 1296)
	('gretathunberg', 1268)
	('peoplenotprofit', 1199)
	('nomoreemptypromises', 1166)
	('uprootthesystem', 1129)
	('cop26', 1109)
	('klimawandel', 1091)
	('climateactionnow', 1083)
	('climate', 1043)


In [33]:
nodes = []
others = []
for i in range(len(sorted_degree)):
    if i<10000:
        nodes.append(sorted_degree[i][0])
        #print("N.nodi:",nodes)
for i in range(len(sorted_degree)):
    if i<3000:
        others.append(sorted_degree[i][0])

##### Abbiamo 15k nodi

In [34]:
len(nodes)

9924

In [35]:
new_fff = fff[(fff["Source"].isin(nodes)) & (fff["Target"].isin(nodes))] 

In [36]:
new_df.to_csv("../Dataset FFF/edges_10K.csv",  index=False)
others_df.to_csv("../Dataset FFF/edges_3K.csv",  index=False)

In [37]:
new_df = pd.DataFrame({"Id": nodes})
new_df.to_csv("../Dataset FFF/nodes_10K.csv", index=False)

In [38]:
others_df = pd.DataFrame({"Id": others})
others_df.to_csv("../Dataset FFF/nodes_3K.csv", index=False)

In [39]:
edges=pd.read_csv('../Dataset FFF/edges_10k.csv')

## Risultato pulizia:

In [40]:
print(nx.info(G))

Graph with 9924 nodes and 182594 edges


In [52]:
new_df

,Id
0,fridaysforfuture
1,climatestrike
2,climatecrisis
3,climateaction
4,climateemergency
...,...
9995,mietwucher
9996,ldk22nrw
9997,happyearthday2022
9998,smallworldafterall


In [54]:
others_df

,Id
0,fridaysforfuture
1,climatestrike
2,climatecrisis
3,climateaction
4,climateemergency
...,...
2995,meatloaf
2996,emergenzaclimatica
2997,aktivisten
2998,happynewyear
